# Ramsey T2
## T2 = {qubit_T2} µs

In [ ]:
f = h5py.File('{HDF5_PATH}', 'r', swmr=True)

wait  = np.unique(f['data']['wait'][()])*4
wait  = wait[wait>0]
data1 = f['data']['I1'][()] + 1j * f['data']['Q1'][()]
data2 = f['data']['I2'][()] + 1j * f['data']['Q2'][()]
data  = data1-data2
data  = IQ_rot(data[0])
data /= np.max(np.abs(data))
if data[0] < 0:
    data *= -1

f.close()

def exp_decay(t, t2, delta, c):
    t2 = np.abs(t2)
    return np.exp(-t/t2) * np.cos(2*np.pi*t*np.abs(delta) + c)

popt, pcov = opt.curve_fit(exp_decay, wait, np.real(data), ({T2}, max({DELTA}, 1e-5), 1e-5))
print('t2, delta, c =', popt)

fig, ax = plt.subplots()
wait_fine = np.linspace(wait[0], wait[-1], int(1e3))

ax.plot(wait, np.real(data), '.-')
ax.plot(wait_fine, exp_decay(wait_fine, *popt), 'k--')
ax.legend(('Data', f'Fit: T2 = {abs(popt[0])/1000:.1f}µs'));

In [ ]:
_opt = popt[0]
_cov = np.array([[pcov[0,0]]])
_results = {'T2_qubit': abs(popt[0]/1000)}
_results